- notebook/requirements.txt: EDA를 위한 종속성 관리
- autoEDA: llama2로 특수 문자 여부를 판단한 내용

In [1]:
import pandas as pd
import pyarrow as pa
import numpy as np
import string
from collections import Counter
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate

In [ ]:
file_path = '../data/train_dataset/train/dataset.arrow'
with pa.memory_map(file_path, 'r') as source:
    table = pa.ipc.open_stream(source).read_all()
df: pd.DataFrame = table.to_pandas()
df.head()

\* 로컬 Ollama 서버로 llama2 활용하는 방법 (macOS 기준)
1. Ollama 설치
    ```
    brew install ollama
    ```
2. Ollama 서버 실행
    ```
    ollama run llama2
    ```
3. Ollama 서버 중단
    ```
    pkill ollama
    ```

In [3]:
llm = OllamaLLM(model="llama2")

In [4]:
prompt_template = """
다음 문서를 분석하여 주어진 질문에 **오직 한국어로**, 가능한 한 짧고 간결한 **단일 구절**로 답변해 주세요. 문장은 사용하지 마세요.

### 문서:
{context}

### 질문:
{question}

답변은 단일 구절이어야 하며, 문장이 되어서는 안 됩니다. 오직 한국어로만 작성해 주세요.
"""

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [10]:
def normalize_korean_answer(s):
    def remove_punctuation(text):
        exclude = set(string.punctuation + "·、，．？！＂＇〃《》「」『』〔〕“”‘’〈〉【】()[]{}")
        return "".join(ch for ch in text if ch not in exclude)

    def white_space_fix(text):
        return " ".join(text.split())

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punctuation(lower(s)))

def exact_match(prediction, ground_truth):
    return normalize_korean_answer(prediction) == normalize_korean_answer(ground_truth)

# F1 Score calculation
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_korean_answer(prediction).split()
    ground_truth_tokens = normalize_korean_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())

    if num_same == 0:
        return 0.0

    precision = num_same / len(prediction_tokens)
    recall = num_same / len(ground_truth_tokens)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [ ]:
def recursive_answer_generation(context, question, current_answer="", depth=0, max_depth=5):
    if depth >= max_depth:
        return current_answer  # 최대 재귀 깊이에 도달하면 종료

    try:
        # 새로운 답변 생성
        result = llm.generate(
            prompts=[prompt.format(context=context, question=question)]
        )
        temp_answer = result.generations[0][0].text.strip()

        # 새로 생성된 답변이 이전 답변과 같으면 재귀 종료
        if temp_answer == current_answer:
            return temp_answer
        
        # 새로 생성된 답변을 사용하여 다시 재귀 호출
        return recursive_answer_generation(temp_answer, question, current_answer=temp_answer, depth=depth + 1)
    
    except Exception as e:
        print(f"Error during recursion at depth {depth}: {e}")
        return current_answer  # 오류 발생 시 현재 답변 반환

exact_matches = []
f1_scores = []

for idx in range(len(df)):
    context = df.iloc[idx]['context']
    question = df.iloc[idx]['question']
    answer = df.iloc[idx]['answers']['text'][0]  # 정답 추출

    predicted_answer = recursive_answer_generation(context, question)        

    # 정확도 평가 (Exact Match와 F1 Score)
    em = exact_match(predicted_answer, answer)
    f1 = f1_score(predicted_answer, answer)

    # 메트릭 저장
    exact_matches.append(em)
    f1_scores.append(f1)
    
    # 출력
    print(f"{idx}.")
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"True Answer: {answer}")
    print(f"Exact Match: {em}, F1 score: {f1}")
    print("-" * 40)

# 전체 평균 메트릭 계산
avg_em = np.mean(exact_matches)
avg_f1 = np.mean(f1_scores)

print(f"Average Exact Match: {avg_em:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")